<img src="https://upload.wikimedia.org/wikipedia/commons/c/c7/HEIG-VD_Logo_96x29_RVB_ROUGE.png" alt="HEIG-VD Logo" width="250" /> 

# Cours TAL - Laboratoire 2
# Mise en œuvre et évaluation de *POS taggers* pour le français

**Objectif**

Appliquer des étiqueteurs morphosyntaxiques (POS taggers) disponibles dans NLTK et dans les outils Stanford NLP à des textes français, puis quantifier leurs performances.

**Instructions initiales**

* Télécharger l'archive `UD_French-GSD-withBlankLines.zip` fournie sur Cyberlearn.
* Placer les trois fichiers qu'elle contient dans le même dossier que le notebook.
* Ce sont des textes en français annotés avec les POS tags, provenant du projet ([Universal Dependencies](https://github.com/UniversalDependencies/UD_French-GSD)), et légèrement modifiés.
  - le fichier `fr-ud-train.conllu3` est destiné à l'entraînement
  - le fichier `fr-ud-dev.conllu3` est destiné aux tests préliminaires et aux réglages des paramètres
  - le fichier `fr-ud-test.conllu3` est destiné à l'évaluation finale.

**Questions préliminaires**

* En inspectant les fichiers, veuillez indiquer le numéro de la colonne où se trouvent les mots, et celui de la colonne où se trouvent leur étiquettes morpho-syntaxiques (*POS tags*).
* Veuillez chercher sur le Web la liste des *POS tags* du projet Universal Dependencies, avec leurs définitions, et indiquer l'URL ci-dessous.


> Réponses: 
> 1. Les mots se trouve dans 2e colonne et leurs étiquettes se trouvent dans 4e colonnes.
> 2. [https://universaldependencies.org/u/pos/](https://universaldependencies.org/u/pos/)

* Veuillez déterminer et afficher le nombre de tokens de chacun des trois fichiers.

In [17]:
# Veuillez écrire votre code ci-dessous, puis exécuter cette cellule.
# since each line of a `conllu3` file corresponds to a single token, 
# we simply need to count each line in the file to find the number of words in the original text

for file in ['fr-ud-test.conllu3', 'fr-ud-dev.conllu3', 'fr-ud-train.conllu3']:
    with open(file, 'r', encoding='utf8') as fd:
        lines = fd.read().split("\n")
        print("Number of tokens in {}: {}".format(file, len(list(filter(lambda x: x != "", lines)))))
        print("Number of phrases in {}: {}".format(file, len(list(filter(lambda x: x == "", lines)))))
        print()

Number of tokens in fr-ud-test.conllu3: 10298
Number of phrases in fr-ud-test.conllu3: 417

Number of tokens in fr-ud-dev.conllu3: 36830
Number of phrases in fr-ud-dev.conllu3: 1479

Number of tokens in fr-ud-train.conllu3: 366371
Number of phrases in fr-ud-train.conllu3: 14555



## Partie 1 : Évaluer le Stanford POS tagger avec les modèles fournis pour le français

L'Université de Stanford fournit un étiqueteur morpho-syntaxique (POS tagger) qui utilise l'apprentissage automatique (https://nlp.stanford.edu/software/tagger.html) appelé Maxent Tagger.  Le tagger et ses modèles multilingues peuvent être téléchargés à l'URL ci-dessus (archive ZIP suivant le lien *Download > full Stanford Tagger version 3.9.2*, 130 MB environ).  

Pour simplifier, on vous propose de télécharger séparément le programme Java [stanford-postagger.jar](https://drive.switch.ch/index.php/s/hMY6yO7lmoQJuS3) et le modèle français [french-ud.tagger](https://drive.switch.ch/index.php/s/4HSqKRTTTkCgPfB) fournis par l'enseignant (mot de passe = reference).  Enregistrez ces deux fichiers dans le même dossier que ce notebook.

Le Maxent Tagger est en Java, et peut être exécuté depuis ce notebook avec un appel Java en ligne de commande.  Pour exécuter une commande système depuis le notebook, ajouter '!' devant (par exemple `! dir` ou `! ls`).  Utilisez la [documentation du Maxent Tagger](https://nlp.stanford.edu/nlp/javadoc/javanlp/edu/stanford/nlp/tagger/maxent/MaxentTagger.html), et plus précisément la section *Tagging and Testing from the command line*, pour comprendre comment l'invoquer.  Java doit être installé sur votre système, et si nécessaire, exécuter :
```python
import os
java_path = 'C:/Program Files (x86)/Java/jdk1.8.0_20/bin/java.exe'  # votre chemin de java.exe
os.environ['JAVA_HOME'] = java_path   # attention aux slash (pas backslash sous Windows)
```
*Note* : il est également possible d'appeler ce tagger avec des commandes NLTK grâce au module [nltk.tag.stanford](https://www.nltk.org/_modules/nltk/tag/stanford.html) mais la gestion des *paths* entre Java, les classes et les modèles peut être compliquée.

In [2]:
import os
java_path = '/usr/bin/java'
os.environ['JAVA_HOME'] = java_path

**Question**

Appliquez le Maxent Tagger pour étiqueter le fichier `fr-ud-dev.conllu3` et demandez à Maxent Tagger de mesurer la qualité par comparaison à une l'annotation de référence fournie dans le fichier. Quels sont les scores obtenus ?  Quel est le nombre le plus important?  Indiquez ces réponses en commentaires du code.

In [3]:
# Resulats:
# Nombre de phrases total correct: 144 (9.742896%); fausse: 1334 (90.257104%).
# Nombre de tags total correct: 32360 (87.863155%); fausse: 4470 (12.136845%).
# Nombre de mots inconnus total correct: 2232 (73.204329%); fausse: 817 (26.795671%).
# 
# Le nombre le plus important est celui lié au nombre de mots *inconnus* qu'il a pu classifier

!java -cp stanford-postagger.jar edu.stanford.nlp.tagger.maxent.MaxentTagger -model 'french-ud.tagger' -testFile 'format=TSV,wordColumn=1,tagColumn=3,fr-ud-dev.conllu3' -verboseResults false

Loading default properties from tagger french-ud.tagger
Loading POS tagger from french-ud.tagger ... done [0.6 sec].
Tagged 36830 words at 16310.89 words per second.
Model french-ud.tagger has xSize=304855, ySize=18, and numFeatures=104853.
Results on 1478 sentences and 36830 words, of which 3049 were unknown.
Total sentences right: 144 (9.742896%); wrong: 1334 (90.257104%).
Total tags right: 32360 (87.863155%); wrong: 4470 (12.136845%).
Unknown words right: 2232 (73.204329%); wrong: 817 (26.795671%).


De même, appliquez le Maxent Tagger pour étiqueter le fichier `fr-ud-test.conllu3` et indiquez la précision du tagger en commentaires du code (#).

In [4]:
# Précisions:
# Phrases: ~12%
# Tokens: ~87%
# Mots inconnus: ~70%
!java -cp stanford-postagger.jar edu.stanford.nlp.tagger.maxent.MaxentTagger -model 'french-ud.tagger' -testFile 'format=TSV,wordColumn=1,tagColumn=3,fr-ud-test.conllu3' -verboseResults false

Loading default properties from tagger french-ud.tagger
Loading POS tagger from french-ud.tagger ... done [0.5 sec].
Tagged 10298 words at 11279.30 words per second.
Model french-ud.tagger has xSize=304855, ySize=18, and numFeatures=104853.
Results on 416 sentences and 10298 words, of which 697 were unknown.
Total sentences right: 54 (12.980769%); wrong: 362 (87.019231%).
Total tags right: 8960 (87.007186%); wrong: 1338 (12.992814%).
Unknown words right: 487 (69.870875%); wrong: 210 (30.129125%).


**Question subsidiare** : combien de phrases et de mots le tagger trouve-t-il dans les fichiers `fr-ud-dev.conllu3` et `fr-ud-test.conllu3` ?  Comparez avec votre propre estimation du nombre de mots.

In [ ]:
# Nous avons trouver 10298 tokens & 417 phrases dans le fichier `fr-ud-test.conllu3` 
#  et le tagger lui en a trouvé 10298 tokens & 416 phrases
# 
# Nous avons trouver 36830 tokens & 1479 phrases dans le fichier `fr-ud-tedevst.conllu3` 
#  et le tagger lui en a trouvé 36830 tokens & 1478 phrases

# Nous avons une différence de 1 sur le nombre de phrase qui est 
#  surement du à une ligne vide supplémentaire à la fin du fichier.

## Partie 2 : Entraîner le Stanford POS tagger pour obtenir de nouveaux modèles

Le but de cette partie est d'entraîner le Maxent Tagger sur les données UD en français (`fr-ud-train.conllu3`), puis de comparer le modèle obtenu avec les modèles fournis par Stanford pour le français, testés dans la partie 1A.  

Suivre la [documentation de Maxent Tagger](https://nlp.stanford.edu/nlp/javadoc/javanlp/edu/stanford/nlp/tagger/maxent/MaxentTagger.html) pour l'entraîner sur le fichier `fr-ud-train.conllu3` et le tester sur `fr-ud-test.conllu3`.  Regardez la section *Training from the command line*. 

La configuration du système pour effectuer l'entraînement est donnée dans un fichier texte, qui peut être produit en suivant la documentation (option `-genprops` pour obtenir un template qui sera édité), soit en s'inspirant du fichier [french-ud.tagger.props](https://drive.switch.ch/index.php/s/gHlam9S74HG2Q4X) accompagnant le modèle `french-ud.tagger` que vous avez utilisé ci-dessus.  Pensez à donner un nouveau nom à votre fichier modèle.

**Questions**

* Créez un fichier `myFrench-ud.tagger.props` qui aboutit à un bon entraînement.  Vous pourrez expérimenter plusieurs fois et proposer le meilleur fichier.  Citez dans le notebook les paramètres sur lesquels vous avez agi.

* Lancez l'entraînement sur le fichier `fr-ud-train.conllu3` (s'il ne tient pas en mémoire, utilisez seulement `fr-ud-dev.conllu3`). Pendant l’entraînement (> 10 minutes, 500 itérations), regardez la suite du travail.

* Évaluez votre modèle comme ci-dessus (sur `dev` et sur `test`).  Quel modèle est meilleur, le vôtre ou celui fourni par Stanford ?  Formulez une hypothèse expliquant ce résultat. 

In [ ]:
# Veuillez écrire votre code ci-dessous, puis exécuter cette cellule.


In [ ]:
# Veuillez écrire votre code ci-dessous, puis exécuter cette cellule.


In [ ]:
# Veuillez écrire votre code ci-dessous, puis exécuter cette cellule.


## Partie 3 : entraîner un POS tagger pour le français dans NLTK

Le but de cette partie est d'utiliser le POS tagger *Averaged Perceptron* de NLTK, en l'entraînant pour le français sur les mêmes données que ci-dessus.  

Notez que pour l'anglais, des taggers pré-entraînés sont disponibles dans NLTK, comme expliqué au [Chapitre 5.1 du livre NLTK](http://www.nltk.org/book/ch05.html) : on peut écrire `nltk.pos_tag(sentence)` où *sentence* est une phrase tokenisée. L'étiquetage morpho-syntaxique produira des paires ('mot', 'TAG').

**Première étape**

Importer les textes annotés `fr-ud-XXXX.conllu3` grâce à des objets `ConllCorpusReader`.  Consultez le mode d'emploi de cette classe directement dans [son code source](https://www.nltk.org/_modules/nltk/corpus/reader/conll.html#ConllCorpusReader), pour déterminer comment lire un fichier en créant un objet `ConllCorpusReader`.  Chargez les trois fichiers, dans trois objets appelés `train_corpus`, `dev_corpus` et `test_corpus`.

In [18]:
from nltk.corpus.reader.conll import ConllCorpusReader

In [19]:
# Veuillez écrire votre code ci-dessous, puis exécuter cette cellule.
root = "."

train_corpus = ConllCorpusReader(
    root, 
    "fr-ud-train.conllu3", 
    ('ignore', 'words', 'ignore', 'pos'),
    separator="\t"
)

dev_corpus = ConllCorpusReader(
    root, 
    "fr-ud-dev.conllu3", 
    ('ignore', 'words', 'ignore', 'pos'),
    separator="\t"
)

test_corpus = ConllCorpusReader(
    root, 
    "fr-ud-test.conllu3", 
    ('ignore', 'words', 'ignore', 'pos'),
    separator="\t"
)

print(train_corpus.tagged_words())
print(dev_corpus.tagged_words())
print(test_corpus.tagged_words())


[('Les', 'DET'), ('commotions', 'NOUN'), ...]
[('Aviator', 'PROPN'), (',', 'PUNCT'), ('un', 'DET'), ...]
[('Je', 'PRON'), ('sens', 'VERB'), ("qu'", 'SCONJ'), ...]


Affichez le nombre de phrases et le nombre de mots de chaque corpus chargé. Cesc chiffres sont-ils identiques à ceux obtenus pour `dev`et pour `test` à la fin de la Partie 1 ?  On peut obtenir les listes de mots étiquetés avec `tagged_words()` et les listes de phrases avec mots étiquetés avec `tagged_sents()`.

In [20]:
print("Training corpus:")
print("Number of words: ", len(train_corpus.tagged_words()))
print("Number of sentences: ", len(train_corpus.tagged_sents()))
print()
print("Dev corpus:")
print("Number of words: ", len(dev_corpus.tagged_words()))
print("Number of sentences: ", len(dev_corpus.tagged_sents()))
print()
print("Testing corpus:")
print("Number of words: ", len(test_corpus.tagged_words()))
print("Number of sentences: ", len(test_corpus.tagged_sents()))
print()

# Nous avons obtenu des chiffres identiques/très proches que précédemment.
# La seul différence proviens du nombre de phrases dans les corpus. Nous avons de nouveau une différence de
# 1 sur le nombre de phrases. (i.e. la même chose que le tagger de Stanford)
#
# Comme dit précédemment, nous pensons que cette différence provient de la ligne vide à la fin du document
# et que les tagger l'ignore.

Training corpus:
Number of words:  366371
Number of sentences:  14554

Dev corpus:
Number of words:  36830
Number of sentences:  1478

Testing corpus:
Number of words:  10298
Number of sentences:  416



Affichez la 17e phrase du corpus de développement (avec les étiquettes POS), et les mots 1001 à 1050 du corpus de test (aussi avec leurs POS tags).

In [21]:
# print("17th sentence with POS", dev_corpus.tagged_sents()[17])

print("17th sentence")
print(dev_corpus.tagged_sents()[17])

print()

print("Words 1001 to 1050")
print(test_corpus.tagged_words()[1001:1050])

17th sentence
[('Johnson', 'PROPN'), ("s'", 'PRON'), ('appuyait', 'VERB'), ('sur', 'ADP'), ('une', 'DET'), ('unique', 'ADJ'), ('forme', 'NOUN'), ('de', 'ADP'), ('rhétorique', 'NOUN'), (',', 'PUNCT'), ('et', 'CCONJ'), ('sa', 'DET'), ('«', 'PUNCT'), ('réfutation', 'NOUN'), ('»', 'PUNCT'), ('de', 'ADP'), ("l'", 'DET'), ('immatérialisme', 'NOUN'), ('de', 'ADP'), ('George', 'PROPN'), ('Berkeley', 'PROPN'), ('est', 'AUX'), ('restée', 'VERB'), ('célèbre', 'ADJ'), (':', 'PUNCT'), ('Berkeley', 'PROPN'), ('affirmait', 'VERB'), ('que', 'SCONJ'), ('la', 'DET'), ('matière', 'NOUN'), ("n'", 'ADV'), ('existait', 'VERB'), ('pas', 'ADV'), ('mais', 'CCONJ'), ('semblait', 'VERB'), ('seulement', 'ADV'), ('exister', 'VERB'), (';', 'PUNCT')]

Words 1001 to 1050
[('la', 'DET'), ('raison', 'NOUN'), ('politique', 'ADJ'), ('...', 'PUNCT'), ('Mais', 'CCONJ'), ('la', 'DET'), ('réalité', 'NOUN'), ('est', 'VERB'), ('que', 'SCONJ'), ('la', 'DET'), ('Mauritanie', 'PROPN'), ("n'", 'ADV'), ('est', 'AUX'), ('pas', 'ADV'

**Seconde étape**

Vous allez maintenant entraîner (sur le corpus `train`) le POS tagger appelé *Averaged Perceptron* fourni par NLTK mais [implémenté par Mathew Honnibal de Explosion.AI](https://explosion.ai/blog/part-of-speech-pos-tagger-in-python).

Dans le [package de NLTK avec des taggers](http://www.nltk.org/api/nltk.tag.html), considérez le module `nltk.tag.perceptron`, pour lequel NLTK explique de façon précise l'entraînement (voir *train the model*) et le test.  Vous allez mettre en oeuvre ces étapes pour entraîner le tagger.  Notez que le modèle est enregistré dans un fichier qui doit finir par `.pickle`, et qui est écrasé à chaque entraînement si vous ne changez pas de nom.  Un modèle peut être également chargé dans un tagger.

In [22]:
# import os # si nécessaire
# import nltk # si nécessaire
# nltk.download('averaged_perceptron_tagger') # si nécessaire
import time
from nltk.tag.perceptron import PerceptronTagger

In [23]:
ptagger = PerceptronTagger(load=False)

Entraînez ici le tagger sur les données d'entraînement, avec les meilleurs paramètres possibles.

In [24]:
# Veuillez écrire votre code ci-dessous, puis exécuter cette cellule.
start_time = time.time()

corpus = [
    [('today','NN'),('is','VBZ'),('good','JJ'),('day','NN')],
    [('yes','NNS'),('it','PRP'),('beautiful','JJ')]
]
print("--- staring training ---")
ptagger.train(train_corpus.tagged_sents(), save_loc="training_result.pickle")
print("--- execution time: %s seconds ---" % (time.time() - start_time))

--- staring training ---
--- execution time: 117.54656457901001 seconds ---


Combien de temps prend l'entraînement ?  Quelle est la taille du fichier modèle résultant ?

In [ ]:
# Execution time: ~117 seconds
# Generated file size: 6.1MB

Évaluez le tagger, d'abord sur les données `dev` puis sur les données `test`.

In [25]:
# Veuillez écrire votre code ci-dessous, puis exécuter cette cellule.
print("Evaluating tagger on dev data:")
print("------------------------------")
print("Score: ", ptagger.evaluate(dev_corpus.tagged_sents()))

print()

print("Evaluating tagger on test data:")
print("------------------------------")
print("Score: ", ptagger.evaluate(test_corpus.tagged_sents()))

Evaluating tagger on dev data:
------------------------------
Score:  0.9657073038284008

Evaluating tagger on test data:
------------------------------
Score:  0.9583414255195184


Veuillez remplir le tableau suivant avec la synthèse des résultats.

| Corpus | MaxEnt      | MaxEnt   | Avg Perceptron   | 
|--------|-------------|----------|------------------|
| -      | fourni      | entraîné | entraîné         |
| dev    |   ~87.86%   |   ..     |  ~96.56%         |
| test   |   ~87.00%   |   ..     |  ~95.78%         |

**Comment se comparent les deux POS taggers sur le français ?  Écrivez vos conclusions dans cette cellule.**  
En regardant le tableau ci-dessus, on voit que les deux POS tagger sont plus efficace que celui de `Stanford`. Nous pensons que cette différence et lié à l'entraînement que ces différents tagger on effectués. Celui de `Stanford` est sûrement entraîné pour tagger n'importe quel texte en français (i.e. il est plus générique) tandis que les deux autres, on était entraîné sur des corpus très similaire les uns des autres.

## Fin du laboratoire 2  

Merci de nettoyer votre feuille, exécuter une dernière fois toutes les instructions, sauvegarder le résultat, et le rendre via Cyberlearn.